<a href="https://colab.research.google.com/github/it-ces/PUJ-GeneticAnnealingNN/blob/main/project/Results-feature-selection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/it-ces/PUJ-GeneticAnnealingNN.git

Cloning into 'PUJ-GeneticAnnealingNN'...
remote: Enumerating objects: 101, done.
remote: Counting objects: 100% (101/101), done.
remote: Compressing objects: 100% (92/92), done.
remote: Total 101 (delta 48), reused 41 (delta 8), pack-reused 0 (from 0)
Receiving objects: 100% (101/101), 3.69 MiB | 7.89 MiB/s, done.
Resolving deltas: 100% (48/48), done.


In [2]:
!pip install deap

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.6/135.6 kB 2.9 MB/s eta 0:00:00


In [3]:
cd PUJ-GeneticAnnealingNN/project

/content/PUJ-GeneticAnnealingNN/project


In [4]:
# Results feature selection
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from preprocessing import complete_vars
from preprocessing import ratios
from preprocessing import breakdown_vars
from preprocessing import dummies_ohe
from preprocessing import Xy
from preprocessing import std_z
from imblearn.under_sampling import RandomUnderSampler
from Genfet import Geafet
from gentools import PerformanceTest
from gentools import get_names
from ineq import Theil
from gentools import PerformanceTest
from gentools import get_names

df_train = pd.read_csv("Datapooled.csv")

VARS = ['Ganancia bruta', 'Ganancia (pérdida)','Ingresos de actividades ordinarias' , 'Costo de ventas', 'Patrimonio total',
     'Total pasivos', 'Total de activos', 'Ganancias acumuladas',  'Pasivos corrientes totales',  'Activos corrientes totales']

df_train.rename(columns={'Clasificación Industrial Internacional Uniforme Versión 4 A.C':'Sector'}, inplace=True)

df_train = df_train[VARS+[ 'event', 'Sector']]
print(df_train[df_train['event']==1].info())
df_train['complete-vars'] = complete_vars(df_train) #1 is that have all variables!
df_train =  df_train[df_train['complete-vars']==1] #filtering firms that have not financial information
print(df_train[df_train['event']==1].info())
df_train = ratios(df_train)
predictors =[ 'GPM', 'NPM', 'ROE','ROA', 'IR', 'DER', 'RSL', 'CR', 'Ax1', 'Ax2', 'Sector']
print(df_train[df_train['event']==1].info())
df_train.replace([np.inf,-np.inf], np.nan, inplace=True)
df_train.dropna(inplace=True)
df_train.drop(columns=['complete-vars'], inplace=True)
df_train = df_train[predictors + ['event']].copy()
df_exp = df_train.copy()
X, y = Xy(df_train, 'event')
cat, binaries, nonormal, normal  = breakdown_vars(X)
nums = nonormal + normal
X = dummies_ohe(X, cat)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle = True, random_state = 666, stratify=y)
rus = RandomUnderSampler(random_state=123)
X_train, y_train = rus.fit_resample(X_train, y_train)
from preprocessing import standardize_X_test
X_test = standardize_X_test(X_train, X_test) # Apply the mean and std of X_test with info from X_train
X_train = std_z(nonormal + normal, X_train)

<class 'pandas.core.frame.DataFrame'>
Index: 771 entries, 4 to 26499
Data columns (total 12 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   Ganancia bruta                      353 non-null    float64
 1   Ganancia (pérdida)                  353 non-null    float64
 2   Ingresos de actividades ordinarias  353 non-null    float64
 3   Costo de ventas                     330 non-null    float64
 4   Patrimonio total                    353 non-null    float64
 5   Total pasivos                       353 non-null    float64
 6   Total de activos                    353 non-null    float64
 7   Ganancias acumuladas                351 non-null    float64
 8   Pasivos corrientes totales          352 non-null    float64
 9   Activos corrientes totales          353 non-null    float64
 10  event                               771 non-null    float64
 11  Sector                              771 non-null

<ipython-input-4-dc126eb85cbe>:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train['complete-vars'] = complete_vars(df_train) #1 is that have all variables!
/usr/local/lib/python3.11/dist-packages/scipy/stats/_axis_nan_policy.py:586: UserWarning: scipy.stats.shapiro: For N > 5000, computed p-value may not be accurate. Current N is 16800.
  res = hypotest_fun_out(*samples, **kwds)


GPM
NPM
ROE
ROA
IR
DER
RSL
CR
Ax1
Ax2


In [5]:
X_test.isnull().sum()

,0
GPM,0
NPM,0
ROE,0
ROA,0
IR,0
DER,0
RSL,0
CR,0
Ax1,0
Ax2,0


In [11]:
classifier = LogisticRegression(penalty=None, max_iter=1500, random_state=666)
Pop_sizes  = [45] # 45
generations = [50]
Mutations = [0.05, 0.09]
Tournaments = [3]
Cross = [0.85, 0.95]
probs_mate = [0.5]
probs_mutate = [0.05 , 0.09]
regist= Geafet(
       pop_sizes = Pop_sizes,
       generations = generations,
       mutations = Mutations,
       tournaments_sizes = Tournaments,
       crossovers=Cross,
       mate_indpb = probs_mate,
       mutate_indpb = probs_mutate,
       classifier = classifier,
       ineq_measure = Theil,
       ineq_min  = 0,
       limit_unchanged= 35,
       X_train=X_train,
       y_train=y_train,
       k_folds = 5,
       verbose=True)

0.7121364583290981 9.417244631936717e-05 [1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0] 0
0.720680467741697 0.00010819198741994855 [0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0] 0
0.7225133123511596 2.9212680923510367e-05 [0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0] 0
0.7225830290887666 2.173854513962987e-05 [0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1] 1
0.7227007446783826 1.9025737599209655e-05 [0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0] 2
0.729628950756514 1.5905522704592006e-05 [0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0] 3
0.729628950756514 7.483058037562596e-06 [0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0] 4
0.729628950756514 6.790123900641807e-06 [0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1,

/usr/local/lib/python3.11/dist-packages/deap/creator.py:185: RuntimeWarning: A class named 'FitnessMax' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
/usr/local/lib/python3.11/dist-packages/deap/creator.py:185: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "


0.7121364583290981 9.296204582444218e-05 [1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0] 0
0.720680467741697 0.00010964608385996041 [0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0] 0
0.7225133123511596 2.9212680923510367e-05 [0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0] 0
0.7225830290887666 2.385979416278396e-05 [0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1] 1
0.7227007446783826 1.8495529903930744e-05 [0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0] 2
0.729628950756514 1.864603311594744e-05 [0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0] 3
0.729628950756514 9.770743893989078e-06 [0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0] 4
0.7297670956316955 2.5462345126252204e-05 [1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1

/usr/local/lib/python3.11/dist-packages/deap/creator.py:185: RuntimeWarning: A class named 'FitnessMax' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
/usr/local/lib/python3.11/dist-packages/deap/creator.py:185: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "


0.7132955310810204 0.00015637580002024465 [0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1] 0
0.7302743567830821 6.381660492057839e-05 [1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1] 0
0.7302743567830821 3.075829370180325e-05 [1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1] 0
0.7342981662080582 2.7555531176143028e-05 [0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1] 1
0.7342981662080582 1.4120386326708297e-05 [0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1] 2
0.732120557976465 1.5643936347071492e-05 [0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1] 3
0.732120557976465 8.07408223799724e-06 [0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1] 4
0.7334775926572249 1.0364653751182417e-05 [0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 

...

0.7121364583290981 9.298640111493807e-05 [1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0] 0
0.7161588445679138 0.00010749168732012737 [0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1] 0
0.7193394348780673 3.2447375421475546e-05 [0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1] 1
0.724795918913566 3.7755338316350304e-05 [0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1] 2
0.7266485416350645 1.8977775923591293e-05 [0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0] 3
0.7283441936220839 1.9673549615549002e-05 [0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1] 4
0.7283441936220839 1.4002533832575906e-05 [0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1] 5
0.729383154661045 1.0738859578431552e-05 [0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 

/usr/local/lib/python3.11/dist-packages/deap/creator.py:185: RuntimeWarning: A class named 'FitnessMax' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
/usr/local/lib/python3.11/dist-packages/deap/creator.py:185: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "


0.7132955310810204 0.00015195162644985683 [0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1] 0
0.7302743567830821 6.41125318935951e-05 [1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1] 0
0.7234144256100318 2.818829752514914e-05 [1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1] 0
0.7249811074768242 2.1990617513235785e-05 [1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1] 1
0.7280769504063308 3.786112289090997e-05 [1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0] 2
0.7296355245587453 2.0495881011196362e-05 [0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0] 3
0.7282166002532977 1.9382211743980112e-05 [0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0] 4
0.7319416741885536 5.099843405581406e-06 [1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1,

/usr/local/lib/python3.11/dist-packages/deap/creator.py:185: RuntimeWarning: A class named 'FitnessMax' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
/usr/local/lib/python3.11/dist-packages/deap/creator.py:185: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "


0.7132955310810204 0.0001634968548080418 [0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1] 0
0.7302743567830821 5.982043571692944e-05 [1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1] 0
0.7234144256100318 3.0551897014227836e-05 [1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1] 0
0.7249811074768242 2.2148937620040987e-05 [1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1] 1
0.7280769504063308 3.801589117169432e-05 [1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0] 2
0.7296355245587453 2.254223763421073e-05 [0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0] 3
0.7282166002532977 2.028640557990213e-05 [0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0] 4
0.7319416741885536 5.470081780220501e-06 [1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 

In [12]:
[hyper for hyper in regist]

[(45, 50, 3, 0.05, 0.85, 0.5, 0.05),
 (45, 50, 3, 0.05, 0.85, 0.5, 0.09),
 (45, 50, 3, 0.05, 0.95, 0.5, 0.05),
 (45, 50, 3, 0.05, 0.95, 0.5, 0.09),
 (45, 50, 3, 0.09, 0.85, 0.5, 0.05),
 (45, 50, 3, 0.09, 0.85, 0.5, 0.09),
 (45, 50, 3, 0.09, 0.95, 0.5, 0.05),
 (45, 50, 3, 0.09, 0.95, 0.5, 0.09)]

In [13]:
posible = [regist[hyper][1] for hyper in regist][0]

In [14]:
[regist[hyper][0] for hyper in regist]

[np.float64(0.732657596371882),
 np.float64(0.7322905419849566),
 np.float64(0.7360536520945022),
 np.float64(0.7360536520945022),
 np.float64(0.7357948298143919),
 np.float64(0.7326694906933936),
 np.float64(0.73458020257496),
 np.float64(0.734623619371283)]

In [15]:
np.array([regist[hyper][0] for hyper in regist]).argmax()

np.int64(2)

In [16]:
fittest_  = [regist[hyper][1] for hyper in regist][np.array([regist[hyper][0] for hyper in regist]).argmax()]

In [17]:
X_test.isnull().sum()

,0
GPM,0
NPM,0
ROE,0
ROA,0
IR,0
DER,0
RSL,0
CR,0
Ax1,0
Ax2,0


In [18]:
for sol in regist:
    print(PerformanceTest(solution = regist[sol][1],
                          classifier = classifier,
                          Xtrain=  X_train,
                          ytrain = y_train,
                          Xtest=  X_test,
                          ytest = y_test))

0.10149253731343283
0.0960960960960961
0.102
0.10089020771513353
0.098
0.09889001009081735
0.09578544061302682
0.09596928982725528


In [19]:
PerformanceTest(solution = posible,
                          classifier = classifier,
                          Xtrain=  X_train,
                          ytrain = y_train,
                          Xtest=  X_test,
                          ytest = y_test)

0.10149253731343283

In [20]:
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.metrics import classification_report

# Forward Selection
sfs = SequentialFeatureSelector(estimator=classifier, n_features_to_select='auto', cv=5)
sfs.fit(X_train, y_train)
forwardvar = sfs.get_support()
sfs.get_feature_names_out()


# Backward Selection
sfs = SequentialFeatureSelector(estimator=classifier, n_features_to_select='auto', cv=5, direction='backward')
sfs.fit(X_train, y_train)
backwardvar = sfs.get_support()
sfs.get_feature_names_out()
X_train.columns[sfs.get_support()]


all = [1 for x in X_train.columns]
candidates = [all, fittest_, backwardvar, forwardvar]

# This function assest the performance of the model in testing

for candidate in candidates:
    print(PerformanceTest(solution = candidate,
                         classifier = classifier,
                         Xtrain=  X_train,
                         ytrain=  y_train,
                         Xtest=  X_test,
                         ytest = y_test))

0.09560229445506692
0.102
0.0964824120603015
0.08833922261484099


In [21]:
# Fittest
vars = get_names(fittest_, X_train)
classifier.fit(X_train.loc[:,vars], y_train)
preds = classifier.predict( X_test.loc[:,vars ])
print(classification_report(y_test, preds))
selectGA  = pd.DataFrame(classification_report(y_test, preds , output_dict=True)).iloc[:,0:2]

              precision    recall  f1-score   support

         0.0       0.99      0.73      0.84      3296
         1.0       0.05      0.80      0.10        64

    accuracy                           0.73      3360
   macro avg       0.52      0.76      0.47      3360
weighted avg       0.98      0.73      0.83      3360



In [22]:

# Forward Selection
sfs = SequentialFeatureSelector(estimator=classifier, n_features_to_select='auto', cv=2)
sfs.fit(X_train, y_train)
forwardvar = sfs.get_support()
sfs.get_feature_names_out()
classifier.fit(X_train.loc[:,forwardvar], y_train)
preds_forward = classifier.predict(X_test.loc[:,forwardvar])
print(classification_report(y_test, preds_forward))
forward  = pd.DataFrame(classification_report(y_test, preds_forward , output_dict=True)).iloc[:,0:2]

              precision    recall  f1-score   support

         0.0       0.99      0.73      0.84      3296
         1.0       0.05      0.78      0.10        64

    accuracy                           0.73      3360
   macro avg       0.52      0.76      0.47      3360
weighted avg       0.98      0.73      0.83      3360



In [23]:
# Backward Selection
sfs = SequentialFeatureSelector(estimator=classifier, n_features_to_select='auto', cv=2, direction='backward')
sfs.fit(X_train, y_train)
backwardvar = sfs.get_support()
classifier.fit(X_train.loc[:,backwardvar], y_train)
preds_backward = classifier.predict(X_test.loc[:,backwardvar])
print(classification_report(y_test, preds_backward))
backward  = pd.DataFrame(classification_report(y_test, preds_backward , output_dict=True)).iloc[:,0:2]


              precision    recall  f1-score   support

         0.0       0.99      0.72      0.84      3296
         1.0       0.05      0.75      0.09        64

    accuracy                           0.72      3360
   macro avg       0.52      0.74      0.47      3360
weighted avg       0.98      0.72      0.82      3360



In [24]:
models_tab = pd.concat([selectGA, forward, backward], axis=1)
cols_names =  pd.MultiIndex.from_tuples([('Genetic selection','No-Default'),("Genetic selection",'Deafult'),
              ("Forward selection",'No-Default'),('Forward selection','Default'),
              ("Backward selection",'No-Default'),('Backward selection','Default')])
models_tab.columns  = cols_names
models_tab = models_tab.style.set_table_styles([
   {'selector': 'th','props': [('text-align', 'center')]}]).format(precision=2)
models_tab.to_latex("feature-selection-logistic.tex")
models_tab

In [25]:
vars

Index(['ROE', 'ROA', 'IR', 'DER', 'RSL', 'CR', 'Sector_C', 'Sector_I',
       'Sector_K', 'Sector_L', 'Sector_O', 'Sector_Q', 'Sector_R', 'Sector_U'],
      dtype='object')

In [26]:
X_train.loc[:,forwardvar].columns

Index(['ROE', 'ROA', 'IR', 'RSL', 'Ax1', 'Sector_C', 'Sector_D', 'Sector_F',
       'Sector_H', 'Sector_I', 'Sector_J', 'Sector_O', 'Sector_R', 'Sector_U'],
      dtype='object')

In [27]:
X_train.loc[:,backwardvar].columns

Index(['GPM', 'NPM', 'ROE', 'IR', 'DER', 'RSL', 'CR', 'Ax1', 'Ax2', 'Sector_C',
       'Sector_F', 'Sector_G', 'Sector_J', 'Sector_P', 'Sector_U'],
      dtype='object')